# Logistic Regression Model: Customer Churn Risk Table

In [ ]:
# Step 1: Import libraries, read in the data, introduce some useful functions

# import the big 4
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Import performance metrics
from sklearn import metrics
from time import time

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve
from sklearn.metrics import precision_recall_curve, f1_score

## Import Logistic Regression from sklearn
from sklearn.linear_model import LogisticRegression

# Import ensemble models
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Accurancy, precision, recall function

def apr(y_pred, y_real):
    accuracy = metrics.accuracy_score(y_real, y_pred)
    precision = metrics.precision_score(y_real, y_pred)
    recall = metrics.recall_score(y_real, y_pred)
    f1 = metrics.f1_score(y_real, y_pred)
    
    print(f"Accuracy:{accuracy}")
    print(f"Precision:{precision}")
    print(f"Recall:{recall}")
    print(f"F1:{f1}")
    return accuracy, precision, recall, f1


## Confusion matrix function

def produce_confusion(positive_label, negative_label, cut_off, df, y_pred_name, y_real_name):
    
    #Set pred to 0 or 1 depending on whether it's higher than the cut_off point.
    
    if cut_off != 'binary':      
        df['pred_binary'] = np.where(df[y_pred_name] > cut_off , 1, 0)
    else: 
        df['pred_binary'] = df[y_pred_name]
    
    #Build the CM
    cm = confusion_matrix(df[y_real_name], df['pred_binary'])  
    
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax=ax, fmt='g'); 

    # labels, title, ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('Real labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels([negative_label, positive_label])
    ax.yaxis.set_ticklabels([negative_label, positive_label]);

    print('Test accuracy = ', accuracy_score(df[y_real_name], df['pred_binary']))

    return accuracy_score(df[y_real_name], df['pred_binary'])

df = pd.read_excel("1 - Project Data.xlsx")

dropped_customers = df[df['Total Charges'].str.strip() == '']

# Mapping columns to get binary results
def columns_binary(x):
    if x == 'Yes':
        return 1
    else:
        return 0


In [ ]:
df.info()

In [ ]:
# Step 2: Feature Engineer & Train Test Split (data leakage avoided: no null-handling aggregations)
def feature_eng(df):

    # Drop rows where 'Total Charges' is missing or empty
    # We have 11 rows with empty Total Charges which we decided to drop rather than replace
    df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')  # Ensure 'Total Charges' is numeric
    df = df.dropna(subset=['Total Charges'])  # Drop rows with missing 'Total Charges'

    
    # Drop non-useful columns
    df = df.drop(columns = ['CustomerID', 
                            'Count', 
                            'Country',
                            'Zip Code',
                            'State', 
                            'Lat Long', 
                            'Latitude', 
                            'Longitude', 
                            'Churn Label',
                            'Churn Reason'],
                            axis = 1) # We already have Churn Value which is already a numerical 

    
    # Change yes/no(including: No internet/Phone services) to binary
    df['Senior Citizen'] = df['Senior Citizen'].apply(columns_binary)
    df['Partner'] = df['Partner'].apply(columns_binary)
    df['Dependents'] = df['Dependents'].apply(columns_binary)
    df['Phone Service'] = df['Phone Service'].apply(columns_binary)
    df['Paperless Billing'] = df['Paperless Billing'].apply(columns_binary)

    # Changing columns with only two results to numericals
    df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})

    # Changing columns with 3 results to numericals
    df['Multiple Lines'] = df['Multiple Lines'].map({'No':0, 'Yes':1, 'No phone service':2})
    df['Online Security'] = df['Online Security'].map({'No':0, 'Yes':1, 'No internet service':2})
    df['Device Protection'] = df['Device Protection'].map({'No':0, 'Yes':1, 'No internet service':2})
    df['Online Backup'] = df['Online Backup'].map({'No':0, 'Yes':1, 'No internet service':2})
    df['Tech Support'] = df['Tech Support'].map({'No':0, 'Yes':1, 'No internet service':2})
    df['Streaming TV'] = df['Streaming TV'].map({'No':0, 'Yes':1, 'No internet service':2})
    df['Streaming Movies'] = df['Streaming Movies'].map({'No':0, 'Yes':1, 'No internet service':2})

    
    # OHE columns
    df = pd.get_dummies(df, columns = ['Internet Service'], drop_first = True, prefix = 'Internet Service', dtype=int)
    df = pd.get_dummies(df, columns = ['Contract'], drop_first = True, prefix = 'Contract', dtype=int)
    df = pd.get_dummies(df, columns = ['Payment Method'], drop_first = True, prefix = 'Payment Method', dtype=int)

    # Map cities to unique integers
    city_mapping = {city: idx for idx, city in enumerate(df['City'].unique())}
    df['City'] = df['City'].map(city_mapping)

    return df


featured_df = feature_eng(df)


# Define feature columns for our predictions
feature_cols = ['City', 'Senior Citizen', 'Partner', 'Dependents',
               'Tenure Months', 'Online Security',
               'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
               'Streaming Movies', 'Paperless Billing', 'Monthly Charges',
               'Total Charges', 'Internet Service_Fiber optic',
               'Internet Service_No', 'Contract_One year', 'Contract_Two year',
               'Payment Method_Credit card (automatic)',
               'Payment Method_Electronic check', 'Payment Method_Mailed check'
               ]


X_train, X_test, y_train, y_test = train_test_split(featured_df[feature_cols],
                                                   featured_df['Churn Value'],
                                                   test_size = 0.3,
                                                   random_state = 99,
                                                   stratify = featured_df['Churn Value'])


In [ ]:
# Step 4: Scale some features

columns_to_scale = ['Monthly Charges', 'Total Charges']
scaler = StandardScaler()
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [ ]:
# Step 5: Fit the model on training set 

lr = LogisticRegression(class_weight='balanced', max_iter=5000, random_state=99)
lr.fit(X_train, y_train)

In [ ]:
# Step 6: Find optimal decision boundary for F1 Score (> 0.7 = GOOD)

# Get predicted probabilities
y_pred_proba = lr.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

# Compute F1 scores for all thresholds
f1_scores = 2 * (precision * recall) / (precision + recall)

# Find the threshold that gives the highest F1-score
optimal_idx = f1_scores.argmax()
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal Threshold: {optimal_threshold}")
print(f"Precision: {precision[optimal_idx]}, Recall: {recall[optimal_idx]}, F1: {f1_scores[optimal_idx]}")

In [ ]:
# Step 6: Find optimal decision boundary for ROC-AUC (0.7 to 0.8 = GOOD)
y_pred_proba = lr.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Find the optimal threshold (e.g., Youden's J statistic = TPR - FPR)
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

print(f"Optimal Threshold: {optimal_threshold}")
print(f"True Positive Rate: {tpr[optimal_idx]}, False Positive Rate: {fpr[optimal_idx]}")

In [ ]:
## Step 7: Predict probabilities, store each binary prediction in a column corresponding to its class
X_train[['active', 'churned']] = lr.predict_proba(X_train)
X_train['y_pred'] = np.where(X_train['churned']>.6, 1, 0) # decision boundary reflecting optimal threshold 

In [ ]:
# Step 8: Evaluate model on test 

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Predictions
y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)[:, 1]

# Print metrics
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
def get_results(actual, predicted):
    print("The confusion matrix for your predictions is:")
    print(metrics.confusion_matrix(actual, predicted), "\n")
    print(f'The accuracy of your model is: {metrics.accuracy_score(actual, predicted)}')
    print(f'The recall of your model is: {metrics.recall_score(actual, predicted)}')
    print(f'The precision of your model is: {metrics.precision_score(actual, predicted)}')
    print(f'The F1-score of your model is: {metrics.f1_score(actual, predicted)}')

In [ ]:
# Step 9: Check Performance Metrics
get_results(y_train, X_train['y_pred'])

In [ ]:
# Step 10: Create table of customer ids & their churn probabilities (test)

# Merge y_test (actual churn status) into X_test for clarity
X_test['actual_churn'] = y_test.values  # Add actual churn labels

# Predict probabilities for the feature set
X_test_clean = X_test[feature_cols] #extract feature columns from x_test
y_pred_proba = lr.predict_proba(X_test_clean)[:, 1] #predict probabilities for test

# Add churn probabilities back to the original X_test
X_test['churn_probability'] = y_pred_proba

# Filter for actives (Churn Value = 0)
final_churner = X_test[X_test['actual_churn'] == 0]

final_churner

In [ ]:
# Step 11: Create table of customer ids & their churn probabilities (train)

X_train['actual_churn'] = y_train.values  # Add actual churn labels

# Predict probabilities for the feature set
X_train_clean = X_train[feature_cols] #extract feature columns from x_test
y_pred_prob = lr.predict_proba(X_train_clean)[:, 1] #predict probabilities for 

# Add churn probabilities back to the original X_train
X_train['churn_probability'] = y_pred_prob

# Filter for actives (Churn Value = 0)
total_churners = X_train[X_train['actual_churn'] == 0]

# Sort by predicted churn probability
total_churners.sort_values(by='churn_probability', ascending=False)

# Display the top 20 churners
total_churners.head(20)

In [ ]:
# Step 11: Concatenate the two dataframes
final_table = pd.concat([total_churners, final_churner], axis=0)
final_table.info()

In [ ]:
total_churners.info()

In [ ]:
final_table.head()

In [ ]:
# Step 12: Add Customer ID back in, reset index

og_df = pd.read_excel("1 - Project Data.xlsx")

# Reset index in both DataFrames to align them properly
final_table = final_table.reset_index()
og_df = og_df.reset_index()

# Merge CustomerID back into final table using the index
total_churners_with_id = final_table.merge(og_df[['CustomerID']], 
                                      left_index=True, 
                                      right_index=True)

# Select only CustomerID and churn_probability for a clean output
total_churners_df = total_churners_with_id[['CustomerID', 'churn_probability']]

# Display the clean DataFrame
total_churners_df.head(10) # Show the top 10 for verification

In [ ]:
# Final: export tables to csv/excel 
total_churners_df.to_csv("Customer Churn Risk 2.csv", index=False)

In [ ]:
total_churners_df.to_excel("Customer Churn Risk 2.xlsx", index=False, engine="openpyxl")